In [1]:
import spacy

text = open(r"C:\Users\cody1\OneDrive\Desktop\ITEC-4230\Kaggle-2024\datasets\Official\test.json")

nlp = spacy.load("en_core_web_sm")

doc = nlp(text)

for entity in doc.ents:
    print(entity.label_ +": "+ entity.text)

ValueError: [E1041] Expected a string, Doc, or bytes as input, but got: <class '_io.TextIOWrapper'>

In [3]:
import json
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

# Load data
with open('../datasets/Official/train.json', 'r') as f:
    train_data = json.load(f)

with open('../datasets/Official/test.json', 'r') as f:
    test_data = json.load(f)

# Extract features and labels
X_train = [' '.join(doc['tokens']) for doc in train_data]
y_train = [1 if 'B-' in doc.get('labels', []) else 0 for doc in train_data]  # 1 for PII, 0 for non-PII

X_test = [' '.join(doc['tokens']) for doc in test_data]
y_test = [1 if 'B-' in doc.get('labels', []) else 0 for doc in test_data]  # 1 for PII, 0 for non-PII

# Feature extraction using Bag of Words
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)
X_test_bow = vectorizer.transform(X_test)

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train_bow.toarray(), dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test_bow.toarray(), dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)

# Define the Random Forest model using PyTorch
class RandomForest(nn.Module):
    def __init__(self, input_size):
        super(RandomForest, self).__init__()
        self.input_size = input_size
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc(x))

# Train the Random Forest model
model = RandomForest(input_size=X_train_torch.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
batch_size = 64

for epoch in range(num_epochs):
    for i in range(0, len(X_train_torch), batch_size):
        inputs = X_train_torch[i:i+batch_size]
        targets = y_train_torch[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()

# Evaluate the model
with torch.no_grad():
    model.eval()
    outputs = model(X_test_torch)
    predicted_labels = (outputs >= 0.5).squeeze().cpu().numpy()
    print(classification_report(y_test, predicted_labels))

# Example prediction
example_text = "John Doe's email is john.doe@example.com"
example_features = vectorizer.transform([example_text])
example_features_torch = torch.tensor(example_features.toarray(), dtype=torch.float32)
with torch.no_grad():
    model.eval()
    output = model(example_features_torch)
    predicted_label = (output >= 0.5).squeeze().cpu().numpy()
    print("Predicted label:", "PII" if predicted_label == 1 else "Non-PII")


              precision    recall  f1-score   support

           0       1.00      1.00      1.00        10

    accuracy                           1.00        10
   macro avg       1.00      1.00      1.00        10
weighted avg       1.00      1.00      1.00        10

Predicted label: Non-PII


In [ ]:
import json
import pandas as pd

# Load data
with open('../datasets/Official/train.json', 'r') as f:
    train_data = json.load(f)

# Initialize lists to store extracted data
output_data = []

# Iterate through each document in the training data
for doc in train_data:
    document_id = doc['document']
    full_text = doc['full_text']
    tokens = doc['tokens']
    labels = doc.get('labels', [])

    # Iterate through tokens and labels
    for i, (token, label) in enumerate(zip(tokens, labels)):
        output_data.append({
            'row_id': i,
            'document': document_id,
            'token': token,
            'label': label
        })

# Convert output data to DataFrame
output_df = pd.DataFrame(output_data)

# Display the DataFrame
print(output_df)


         row_id  document       token label
0             0         7      Design     O
1             1         7    Thinking     O
2             2         7         for     O
3             3         7  innovation     O
4             4         7   reflexion     O
...         ...       ...         ...   ...
4992528     815     22687     process     O
4992529     816     22687   explained     O
4992530     817     22687       above     O
4992531     818     22687           .     O
4992532     819     22687        \n\n     O

[4992533 rows x 4 columns]


In [ ]:
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import classification_report

# Load data
with open('../datasets/Official/train.json', 'r') as f:
    train_data = json.load(f)

# Initialize lists to store extracted data
output_data = []

# Extract features and labels
X_train = []
y_train = []

# Iterate through each document in the training data
for doc in train_data:
    document_id = doc['document']
    full_text = doc['full_text']
    tokens = doc['tokens']
    labels = doc.get('labels', [])

    # Iterate through tokens and labels
    for i, (token, label) in enumerate(zip(tokens, labels)):
        X_train.append(' '.join(tokens[:i+1]))  # Use tokens up to the current position
        y_train.append(1 if 'B-' in label else 0)  # 1 for PII, 0 for non-PII

        # Store extracted data
        output_data.append({
            'row_id': i,
            'document': document_id,
            'token': token,
            'label': label
        })

# Convert lists to NumPy arrays
X_train = np.array(X_train)
y_train = np.array(y_train)

# Feature extraction using Bag of Words
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train_bow.toarray(), dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)

# Define the Random Forest model using PyTorch
class RandomForest(nn.Module):
    def __init__(self, input_size):
        super(RandomForest, self).__init__()
        self.input_size = input_size
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc(x))

# Train the Random Forest model
model = RandomForest(input_size=X_train_torch.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10
batch_size = 64

for epoch in range(num_epochs):
    for i in range(0, len(X_train_torch), batch_size):
        inputs = X_train_torch[i:i+batch_size]
        targets = y_train_torch[i:i+batch_size]

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()

# Extracting PII from the test data can be done similarly as done for the training data
# You can also evaluate the model's performance using classification_report as before


MemoryError: Unable to allocate 331. GiB for an array with shape (4992533,) and data type <U17820

In [4]:
# Load data
with open('../datasets/Official/train.json', 'r') as f:
    train_data = json.load(f)

# Extract features and labels
X_train = []
y_train = []

# Iterate through each document in the training data
for doc in train_data:
    document_id = doc['document']
    tokens = doc['tokens']
    labels = doc.get('labels', [])

    # Iterate through tokens and labels
    for i, (token, label) in enumerate(zip(tokens, labels)):
        X_train.append(' '.join(tokens[:i+1]))  # Use tokens up to the current position
        y_train.append(1 if 'B-' in label else 0)  # 1 for PII, 0 for non-PII

# Feature extraction using Bag of Words
vectorizer = CountVectorizer()
X_train_bow = vectorizer.fit_transform(X_train)

# Define batch size
batch_size = 1000

# Define a function to process data in batches
def process_batch(batch_data):
    X_batch = [' '.join(doc['tokens']) for doc in batch_data]
    y_batch = [1 if 'B-' in doc.get('labels', []) else 0 for doc in batch_data]
    X_batch_bow = vectorizer.transform(X_batch)
    return X_batch_bow, np.array(y_batch)

# Process data in batches
X_train_batches = []
y_train_batches = []

num_batches = len(train_data) // batch_size + 1
for i in range(num_batches):
    batch_data = train_data[i*batch_size:(i+1)*batch_size]
    X_batch, y_batch = process_batch(batch_data)
    X_train_batches.append(X_batch)
    y_train_batches.append(y_batch)

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(np.concatenate(X_train_batches), dtype=torch.float32)
y_train_torch = torch.tensor(np.concatenate(y_train_batches), dtype=torch.float32)

# Define the Random Forest model using PyTorch
class RandomForest(nn.Module):
    def __init__(self, input_size):
        super(RandomForest, self).__init__()
        self.input_size = input_size
        self.fc = nn.Linear(input_size, 1)

    def forward(self, x):
        return torch.sigmoid(self.fc(x))

# Train the Random Forest model
model = RandomForest(input_size=X_train_torch.shape[1])
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

for epoch in range(num_epochs):
    for X_batch, y_batch in zip(X_train_batches, y_train_batches):
        inputs = torch.tensor(X_batch.toarray(), dtype=torch.float32)
        targets = torch.tensor(y_batch, dtype=torch.float32)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets.view(-1, 1))
        loss.backward()
        optimizer.step()

# Extracting PII from the test data can be done similarly as done for the training data
# You can also evaluate the model's performance using classification_report as before

ValueError: zero-dimensional arrays cannot be concatenated

# Pre-processing

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import json

# Define the path to the JSON file
json_file_path = '../datasets/Official/train.json'

# Initialize an empty list to store the training data
training_data = []

# Read the JSON file and store the data in the training_data list
with open(json_file_path, 'r') as file:
    training_data = json.load(file)

# Prepare data for preprocessing
documents = []
token_sequences = []
labels_sequences = []

# Iterate over each data object
for data in training_data:
    document = data["document"]
    full_text = data["full_text"]
    tokens = data["tokens"]
    labels = data["labels"]

    # Join tokens into a single string
    token_sequence = " ".join(tokens)
    
    # Join labels into a single string
    label_sequence = " ".join(labels)
    
    # Append to lists
    documents.append(document)
    token_sequences.append(token_sequence)
    labels_sequences.append(label_sequence)

# Initialize and fit CountVectorizer for token sequences
vectorizer = CountVectorizer()
X_token = vectorizer.fit_transform(token_sequences)

# Initialize and fit LabelEncoder for labels
label_encoder = LabelEncoder()
y_label = label_encoder.fit_transform(labels_sequences)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_token, y_label, test_size=0.2, random_state=42)

# You may also need to perform padding, depending on the requirements of your model and library used
# For example, if using PyTorch, you can pad sequences using torch.nn.utils.rnn.pad_sequence
# If using TensorFlow/Keras, you can use tf.keras.preprocessing.sequence.pad_sequences


In [11]:
import json
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

# Step 1: Read training data
with open('../datasets/Official/train.json', 'r') as file:
    train_data = json.load(file)

# Step 2: Preprocess training data
train_texts = []
train_labels = []

for item in train_data:
    text = item['full_text']
    labels = ['O'] * len(item['tokens'])  # Initialize all labels as 'O'
    for label_info in item['labels']:
        print(label_info)  # Add this line for debugging
        label_type = label_info[0]  # Correct access to label type
        start_idx = label_info[1]   # Correct access to start index
        end_idx = label_info[2]     # Correct access to end index
        labels[start_idx:end_idx] = ['B-' + label_type] + ['I-' + label_type] * (end_idx - start_idx - 1)
    train_texts.append(text)
    train_labels.extend(labels)

# Step 3: Train machine learning model
vectorizer = CountVectorizer()
X_train = vectorizer.fit_transform(train_texts)
y_train = train_labels

model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Step 4: Read test data
with open('../datasets/Official/test.json', 'r') as file:
    test_data = json.load(file)

# Step 5: Preprocess test data
test_texts = [item['full_text'] for item in test_data]

# Step 6: Use the trained model to predict labels for test data
X_test = vectorizer.transform(test_texts)
predicted_labels = model.predict(X_test)

# Step 7: Write predicted labels to output file
output_data = []
for idx, item in enumerate(test_data):
    for token_idx, token in enumerate(item['tokens']):
        output_data.append([idx, item['document'], token_idx, predicted_labels.pop(0)])

# Write the output data to a CSV file
output_file = "../datasets/output.csv"
with open(output_file, "w") as file:
    # Write the header
    file.write("row_id,document,token,label\n")
    
    # Write each data row
    for row in output_data:
        file.write(",".join(map(str, row)) + "\n")

print("Output file created:", output_file)


O


IndexError: string index out of range

In [12]:
from sklearn import svm
from sklearn import datasets
# Load dataset
iris = datasets.load_iris()
clf = svm.LinearSVC()
# learn from the data
clf.fit(iris.data, iris.target)
# predict for unseen data
clf.predict([[ 5.0,  3.6,  1.3,  0.25]])
# Parameters of model can be changed by using the attributes ending with an underscore
print(clf.coef_ )

[[ 0.18423669  0.45122683 -0.80794598 -0.45071834]
 [ 0.05537981 -0.90087639  0.40947615 -0.96117093]
 [-0.85066716 -0.98671898  1.381056    1.86525244]]


C:\Users\cody1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
C:\Users\cody1\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\svm\_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
